In [1]:
from glob import glob
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
!ls ../../ICDAR2011/trainingSet/OfflineSignatures/

Chinese  Dutch


In [3]:
def convert2jpg(paths):
    news_paths=[]
    for path in paths:
        img=cv2.imread(path)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(np.array(img),(5,5),0)
        ret3,img = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        savepath=path[:6] + 'SiameseData/' + path[6:-3]+'jpg'
        dirs='/'.join(savepath.split('/')[:-1])
        if os.path.exists(dirs) is False:
            os.makedirs(dirs)
        cv2.imwrite(savepath,img)
        news_paths.append(savepath)
    return news_paths

In [4]:
def traindf(forge,real,file):
    df_forged=pd.DataFrame(zip(forge,['forge']*len(forge)),columns=['forged_path','forged_label'])
    df_real=pd.DataFrame(zip(real,['real']*len(real)),columns=['real_path','real_label'])
    print(len(df_real),len(df_forged))
#     df_forged.head()
    df_real['real_id']=df_real.real_path.apply(lambda x:x.split('_')[0][-2::]).astype(int)
    df_forged['forged_id']=df_forged.forged_path.apply(lambda x:x.split('_')[0][-2::]).astype(int)
    df_real['real_idx']=df_real.real_path.apply(lambda x:x.split("_")[-1].split('.')[0]).astype(int)
    df_forged['forged_idx']=df_forged.forged_path.apply(lambda x:x.split("_")[-1].split('.')[0]).astype(int)
    print(len(df_real),len(df_forged))
    df=pd.merge(left=df_real,right=df_forged,left_on='real_id',right_on='forged_id',how='outer')
    df.drop_duplicates(inplace=True)
    df['data']=file
    print(len(df))
    #print(df.head())
    df.sort_values(by=['forged_id','real_id'],inplace=True)
    df.to_csv('../csv/{}.csv'.format(file),index=False)

In [5]:
chinese_forge=glob('../../ICDAR2011/trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Forgeries/*.png')
chinese_real=glob('../../ICDAR2011/trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Genuine/*.png')
print(len(chinese_forge)+len(chinese_real))#575 signatures
chinese_forge=convert2jpg(chinese_forge)
chinese_real=convert2jpg(chinese_real)
print(len(chinese_forge)+len(chinese_real))#575 signatures
chinese_forge[0]

575
575


'../../SiameseData/ICDAR2011/trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Forgeries/0100003_1.jpg'

In [6]:
chinese_forge[0]

'../../SiameseData/ICDAR2011/trainingSet/OfflineSignatures/Chinese/TrainingSet/Offline Forgeries/0100003_1.jpg'

In [7]:
traindf(chinese_forge,chinese_real,'chinese')

235 340
235 340
7988


In [8]:
#prepare test data

In [9]:
quest1=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Questioned(487)/*/*.png')
quest2=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Questioned(487)/*/*.PNG')
forge=quest2
real1=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Ref(115)/*/*.png')
real2=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Ref(115)/*/*.PNG')
real=real1+real2+quest1
print(len(forge),len(real))
forge=convert2jpg(forge)
real=convert2jpg(real)
print(len(forge)+len(real))#575 signatures
forge[0]

367 234
601


'../../SiameseData/ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Questioned(487)/011/06_0113011.jpg'

In [10]:
def testdf(forge,real,file):
    df_forged=pd.DataFrame(zip(forge,['forge']*len(forge)),columns=['forged_path','forged_label'])
    df_real=pd.DataFrame(zip(real,['real']*len(real)),columns=['real_path','real_label'])
    print(len(df_real),len(df_forged))
    print(df_real.real_path[0].split('/')[-2])
    df_real['real_id']=df_real.real_path.apply(lambda x:x.split('/')[-2]).astype(int)
    df_forged['forged_id']=df_forged.forged_path.apply(lambda x:x.split('/')[-2]).astype(int)
    print(df_real.head().real_path[0])
    df_real['real_idx']=df_real.real_path.apply(lambda x:x.split('_')[1].split('/')[-1]).astype(int)
    df_forged['forged_idx']=df_forged.forged_path.apply(lambda x:x.split('_')[1].split('/')[-1]).astype(int)
    
    print(len(df_real),len(df_forged))
    df=pd.merge(left=df_real,right=df_forged,left_on='real_id',right_on='forged_id',how='outer')
    df.drop_duplicates(inplace=True)
    df['data']=file
    print(len(df))
#     print(df.head())
    df.sort_values(by=['forged_id','real_id'],inplace=True)
    df.to_csv('../csv/test_{}.csv'.format(file),index=False)

In [11]:
testdf(forge,real,'chinese')

234 367
011
../../SiameseData/ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Chinese/Ref(115)/011/13_011.jpg
234 367
8587


In [12]:
forge=glob('../../ICDAR2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Forgeries/*.png')
real=glob('../../ICDAR2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/*.PNG')
print(len(forge),len(real))#575 signatures
forge=convert2jpg(forge)
real=convert2jpg(real)
print(len(forge)+len(real))#575 signatures
forge[0]
traindf(forge,real,'dutch')

123 239


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


362
239 123
239 123
2936


In [13]:
quest1=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Questioned(1287)/*/*.PNG')
real1=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Reference(646)/*/*.PNG')
quest2=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Questioned(1287)/*/*.png')
real2=glob('../../ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Reference(646)/*/*.png')
forge=quest1
real=real1+real2+quest2
print(len(forge),len(real))
forge=convert2jpg(forge)
real=convert2jpg(real)
print(len(forge)+len(real))#575 signatures
forge[0]

639 1294


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

1933


'../../SiameseData/ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Questioned(1287)/044/01_0103044.jpg'

In [14]:
testdf(forge,real,'dutch')

1294 639
041
../../SiameseData/ICDAR2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/Reference(646)/041/13_041.jpg
1294 639
15308


In [15]:
239*123

29397